# I. Import library and load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv("processed data.csv")
data = data.drop(columns = ['Unnamed: 0'])
data.head()

,MP,Min,Goals,Assists,CrdY,CrdR,Comp,Shots,Acceleration,Aggression,...,Vision,Volleys,age,name,overall,potential,preferred_foot,position common,market value,wage value
0,34,2983,0.06,0.00,0.15,0.03,Ligue 1,0.54,37,81,...,45,24,33,Yunis Abdelhamid,76,76,Left,defender,3600000.0,23000.0
1,31,2462,0.04,0.00,0.44,0.11,Ligue 1,0.66,50,79,...,65,38,21,Salis Abdul Samed,70,75,Right,midfielder,2200000.0,7000.0
2,34,2956,0.00,0.06,0.27,0.00,Ligue 1,0.91,77,76,...,66,37,28,Laurent Abergel,75,75,Right,midfielder,4900000.0,18000.0
3,24,726,0.00,0.12,0.37,0.00,Bundesliga,2.22,81,39,...,44,53,22,Dickson Abiama,68,76,Right,striker,2700000.0,9000.0
4,30,2536,0.14,0.00,0.07,0.04,Serie A,0.57,64,77,...,65,39,33,Francesco Acerbi,83,83,Left,defender,17500000.0,75000.0


# II. Convert category data into numeric data and remove columns that don't related in build model

In [3]:
# Comp = name of league
data['Comp'].replace(data['Comp'].unique(),[1, 3, 2, 4, 5], inplace=True)
# drop name column and preferred_foot
data = data.drop(columns = ['name','preferred_foot'])

In [4]:
#convert the categorical variables to dummies
data = pd.get_dummies(data) 
# log tranformation for market vakue
data['market value'] = np.log(data['market value'])
data.head()

,MP,Min,Goals,Assists,CrdY,CrdR,Comp,Shots,Acceleration,Aggression,...,Volleys,age,overall,potential,market value,wage value,position common_defender,position common_goalkeeper,position common_midfielder,position common_striker
0,34,2983,0.06,0.00,0.15,0.03,1,0.54,37,81,...,24,33,76,76,15.096444,23000.0,1,0,0,0
1,31,2462,0.04,0.00,0.44,0.11,1,0.66,50,79,...,38,21,70,75,14.603968,7000.0,0,0,1,0
2,34,2956,0.00,0.06,0.27,0.00,1,0.91,77,76,...,37,28,75,75,15.404746,18000.0,0,0,1,0
3,24,726,0.00,0.12,0.37,0.00,3,2.22,81,39,...,53,22,68,76,14.808762,9000.0,0,0,0,1
4,30,2536,0.14,0.00,0.07,0.04,2,0.57,64,77,...,39,33,83,83,16.677711,75000.0,1,0,0,0


# III. Split training and test

In [5]:
data_build = data.copy()
y = data_build['market value']
x = data_build.drop(columns =['market value'])

# Standardization
def standard_scale(x):
    convert = StandardScaler().fit(x)
    return convert.transform(x)
#x = standard_scale(x)
x_train, x_test, y_train, y_test = train_test_split(x,y , test_size = 0.33, random_state = 3 )

# IV.Emsemble models

## 1.Adaboost

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
DCT = DecisionTreeRegressor(random_state=0, max_depth = 13)
regressor = AdaBoostRegressor(base_estimator = DCT)
params = {'n_estimators': range(1,100,3),'random_state' : [3],
         }
scoring_fnc = 'neg_mean_squared_error'
grid = GridSearchCV(estimator=regressor, param_grid=params, scoring= scoring_fnc, cv= 5 , verbose= 5)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 33 candidates, totalling 165 fits
[CV 1/5] END ...n_estimators=1, random_state=3;, score=-0.085 total time=   0.0s
[CV 2/5] END ...n_estimators=1, random_state=3;, score=-0.071 total time=   0.0s
[CV 3/5] END ...n_estimators=1, random_state=3;, score=-0.077 total time=   0.0s
[CV 4/5] END ...n_estimators=1, random_state=3;, score=-0.048 total time=   0.0s
[CV 5/5] END ...n_estimators=1, random_state=3;, score=-0.101 total time=   0.0s
[CV 1/5] END ...n_estimators=4, random_state=3;, score=-0.037 total time=   0.0s
[CV 2/5] END ...n_estimators=4, random_state=3;, score=-0.045 total time=   0.0s
[CV 3/5] END ...n_estimators=4, random_state=3;, score=-0.050 total time=   0.0s
[CV 4/5] END ...n_estimators=4, random_state=3;, score=-0.039 total time=   0.0s
[CV 5/5] END ...n_estimators=4, random_state=3;, score=-0.044 total time=   0.0s
[CV 1/5] END ...n_estimators=7, random_state=3;, score=-0.034 total time=   0.0s
[CV 2/5] END ...n_estimators=7, random_state=3;

[CV 2/5] END ..n_estimators=61, random_state=3;, score=-0.025 total time=   1.0s
[CV 3/5] END ..n_estimators=61, random_state=3;, score=-0.029 total time=   1.0s
[CV 4/5] END ..n_estimators=61, random_state=3;, score=-0.021 total time=   1.0s
[CV 5/5] END ..n_estimators=61, random_state=3;, score=-0.032 total time=   1.0s
[CV 1/5] END ..n_estimators=64, random_state=3;, score=-0.018 total time=   1.0s
[CV 2/5] END ..n_estimators=64, random_state=3;, score=-0.025 total time=   1.0s
[CV 3/5] END ..n_estimators=64, random_state=3;, score=-0.030 total time=   1.0s
[CV 4/5] END ..n_estimators=64, random_state=3;, score=-0.021 total time=   1.0s
[CV 5/5] END ..n_estimators=64, random_state=3;, score=-0.032 total time=   1.0s
[CV 1/5] END ..n_estimators=67, random_state=3;, score=-0.019 total time=   1.1s
[CV 2/5] END ..n_estimators=67, random_state=3;, score=-0.024 total time=   1.1s
[CV 3/5] END ..n_estimators=67, random_state=3;, score=-0.030 total time=   1.1s
[CV 4/5] END ..n_estimators=

GridSearchCV(cv=5,
             estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=13,
                                                                              random_state=0)),
             param_grid={'n_estimators': range(1, 100, 3), 'random_state': [3]},
             scoring='neg_mean_squared_error', verbose=5)

In [7]:
# get best parameter
grid.best_estimator_.get_params()

{'base_estimator__ccp_alpha': 0.0,
 'base_estimator__criterion': 'squared_error',
 'base_estimator__max_depth': 13,
 'base_estimator__max_features': None,
 'base_estimator__max_leaf_nodes': None,
 'base_estimator__min_impurity_decrease': 0.0,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 2,
 'base_estimator__min_weight_fraction_leaf': 0.0,
 'base_estimator__random_state': 0,
 'base_estimator__splitter': 'best',
 'base_estimator': DecisionTreeRegressor(max_depth=13, random_state=0),
 'learning_rate': 1.0,
 'loss': 'linear',
 'n_estimators': 94,
 'random_state': 3}

In [8]:
#  Training model with hyperparameter tuning
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=13, random_state=0)
                          ,n_estimators = 94, learning_rate = 1.0, random_state = 3)
model.fit(x_train, y_train)
y_pre = model.predict(x_test)
print('Root mean squared' ,(mean_squared_error(y_pre,y_test )**(1/2)))
print('R2-scored ', r2_score(y_pre, y_test))

Root mean squared 0.12748218988490542
R2-scored  0.9894280792870761


In [9]:
t = np.exp(y_pre) - np.exp(y_test)
d = np.exp(y_test)
m = abs(t) / d
sum(m) / len(m)

0.07645676509872117

# 2. XGBoost

In [10]:
from xgboost import XGBRegressor

In [11]:
regressor = XGBRegressor()

params = {
    'n_estimators': range(100, 220, 40),
    
    'learning_rate': [x/100 for x in range(1, 21, 2)],
    'gamma': [x/5 for x in range(0, 3, 1)],
    'max_depth':[int(x) for x in range(3, 17, 2)]
}
scoring_fnc = 'neg_mean_squared_error'
grid = GridSearchCV(estimator=regressor, param_grid=params, scoring= scoring_fnc, cv= 5 , verbose= 5)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 630 candidates, totalling 3150 fits
[CV 1/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estimators=100;, score=-30.509 total time=   0.6s
[CV 2/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estimators=100;, score=-30.947 total time=   0.0s
[CV 3/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estimators=100;, score=-30.927 total time=   0.1s
[CV 4/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estimators=100;, score=-30.639 total time=   0.0s
[CV 5/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estimators=100;, score=-31.723 total time=   0.1s
[CV 1/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estimators=140;, score=-13.761 total time=   0.1s
[CV 2/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estimators=140;, score=-13.970 total time=   0.1s
[CV 3/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estimators=140;, score=-13.952 total time=   0.2s
[CV 4/5] END gamma=0.0, learning_rate=0.01, max_depth=3, n_estim

[CV 1/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=100;, score=-30.505 total time=   0.1s
[CV 2/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=100;, score=-30.945 total time=   0.1s
[CV 3/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=100;, score=-30.928 total time=   0.1s
[CV 4/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=100;, score=-30.643 total time=   0.1s
[CV 5/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=100;, score=-31.717 total time=   0.1s
[CV 1/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=140;, score=-13.756 total time=   0.2s
[CV 2/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=140;, score=-13.971 total time=   0.2s
[CV 3/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=140;, score=-13.954 total time=   0.2s
[CV 4/5] END gamma=0.0, learning_rate=0.01, max_depth=13, n_estimators=140;, score=-13.790 total time=   0.2s
[CV 5/5] E

[CV 2/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=100;, score=-0.570 total time=   0.1s
[CV 3/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=100;, score=-0.571 total time=   0.1s
[CV 4/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=100;, score=-0.555 total time=   0.1s
[CV 5/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=100;, score=-0.600 total time=   0.1s
[CV 1/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=140;, score=-0.063 total time=   0.3s
[CV 2/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=140;, score=-0.065 total time=   0.3s
[CV 3/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=140;, score=-0.068 total time=   0.3s
[CV 4/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=140;, score=-0.062 total time=   0.3s
[CV 5/5] END gamma=0.0, learning_rate=0.03, max_depth=9, n_estimators=140;, score=-0.068 total time=   0.3s
[CV 1/5] END gamma=0.0, lear

[CV 3/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=100;, score=-0.026 total time=   0.2s
[CV 4/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=100;, score=-0.022 total time=   0.2s
[CV 5/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=100;, score=-0.024 total time=   0.2s
[CV 1/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=140;, score=-0.010 total time=   0.3s
[CV 2/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=140;, score=-0.013 total time=   0.3s
[CV 3/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=140;, score=-0.014 total time=   0.3s
[CV 4/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=140;, score=-0.012 total time=   0.3s
[CV 5/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=140;, score=-0.012 total time=   0.3s
[CV 1/5] END gamma=0.0, learning_rate=0.05, max_depth=5, n_estimators=180;, score=-0.010 total time=   0.4s
[CV 2/5] END gamma=0.0, lear

[CV 4/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=100;, score=-0.022 total time=   0.3s
[CV 5/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=100;, score=-0.024 total time=   0.3s
[CV 1/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=140;, score=-0.010 total time=   0.6s
[CV 2/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=140;, score=-0.013 total time=   0.7s
[CV 3/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=140;, score=-0.014 total time=   0.7s
[CV 4/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=140;, score=-0.012 total time=   0.7s
[CV 5/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=140;, score=-0.012 total time=   0.7s
[CV 1/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=180;, score=-0.009 total time=   1.0s
[CV 2/5] END gamma=0.0, learning_rate=0.05, max_depth=15, n_estimators=180;, score=-0.013 total time=   1.0s
[CV 3/5] END gamma=

...

[CV 3/5] END gamma=0.4, learning_rate=0.15, max_depth=9, n_estimators=140;, score=-0.019 total time=   0.6s
[CV 4/5] END gamma=0.4, learning_rate=0.15, max_depth=9, n_estimators=140;, score=-0.016 total time=   0.5s
[CV 5/5] END gamma=0.4, learning_rate=0.15, max_depth=9, n_estimators=140;, score=-0.019 total time=   0.7s
[CV 1/5] END gamma=0.4, learning_rate=0.15, max_depth=9, n_estimators=180;, score=-0.013 total time=   0.7s
[CV 2/5] END gamma=0.4, learning_rate=0.15, max_depth=9, n_estimators=180;, score=-0.018 total time=   0.8s
[CV 3/5] END gamma=0.4, learning_rate=0.15, max_depth=9, n_estimators=180;, score=-0.019 total time=   0.8s
[CV 4/5] END gamma=0.4, learning_rate=0.15, max_depth=9, n_estimators=180;, score=-0.016 total time=   0.7s
[CV 5/5] END gamma=0.4, learning_rate=0.15, max_depth=9, n_estimators=180;, score=-0.019 total time=   0.9s
[CV 1/5] END gamma=0.4, learning_rate=0.15, max_depth=11, n_estimators=100;, score=-0.013 total time=   0.5s
[CV 2/5] END gamma=0.4, lea

[CV 4/5] END gamma=0.4, learning_rate=0.17, max_depth=5, n_estimators=140;, score=-0.017 total time=   0.3s
[CV 5/5] END gamma=0.4, learning_rate=0.17, max_depth=5, n_estimators=140;, score=-0.019 total time=   0.3s
[CV 1/5] END gamma=0.4, learning_rate=0.17, max_depth=5, n_estimators=180;, score=-0.021 total time=   0.4s
[CV 2/5] END gamma=0.4, learning_rate=0.17, max_depth=5, n_estimators=180;, score=-0.018 total time=   0.4s
[CV 3/5] END gamma=0.4, learning_rate=0.17, max_depth=5, n_estimators=180;, score=-0.020 total time=   0.4s
[CV 4/5] END gamma=0.4, learning_rate=0.17, max_depth=5, n_estimators=180;, score=-0.017 total time=   0.4s
[CV 5/5] END gamma=0.4, learning_rate=0.17, max_depth=5, n_estimators=180;, score=-0.019 total time=   0.5s
[CV 1/5] END gamma=0.4, learning_rate=0.17, max_depth=7, n_estimators=100;, score=-0.021 total time=   0.4s
[CV 2/5] END gamma=0.4, learning_rate=0.17, max_depth=7, n_estimators=100;, score=-0.018 total time=   0.3s
[CV 3/5] END gamma=0.4, lear

[CV 5/5] END gamma=0.4, learning_rate=0.17, max_depth=15, n_estimators=140;, score=-0.019 total time=   1.3s
[CV 1/5] END gamma=0.4, learning_rate=0.17, max_depth=15, n_estimators=180;, score=-0.021 total time=   1.8s
[CV 2/5] END gamma=0.4, learning_rate=0.17, max_depth=15, n_estimators=180;, score=-0.018 total time=   1.7s
[CV 3/5] END gamma=0.4, learning_rate=0.17, max_depth=15, n_estimators=180;, score=-0.019 total time=   1.5s
[CV 4/5] END gamma=0.4, learning_rate=0.17, max_depth=15, n_estimators=180;, score=-0.017 total time=   1.8s
[CV 5/5] END gamma=0.4, learning_rate=0.17, max_depth=15, n_estimators=180;, score=-0.019 total time=   1.6s
[CV 1/5] END gamma=0.4, learning_rate=0.19, max_depth=3, n_estimators=100;, score=-0.017 total time=   0.1s
[CV 2/5] END gamma=0.4, learning_rate=0.19, max_depth=3, n_estimators=100;, score=-0.016 total time=   0.1s
[CV 3/5] END gamma=0.4, learning_rate=0.19, max_depth=3, n_estimators=100;, score=-0.023 total time=   0.1s
[CV 4/5] END gamma=0.4

[CV 1/5] END gamma=0.4, learning_rate=0.19, max_depth=11, n_estimators=180;, score=-0.017 total time=   1.0s
[CV 2/5] END gamma=0.4, learning_rate=0.19, max_depth=11, n_estimators=180;, score=-0.014 total time=   1.1s
[CV 3/5] END gamma=0.4, learning_rate=0.19, max_depth=11, n_estimators=180;, score=-0.024 total time=   1.2s
[CV 4/5] END gamma=0.4, learning_rate=0.19, max_depth=11, n_estimators=180;, score=-0.019 total time=   1.2s
[CV 5/5] END gamma=0.4, learning_rate=0.19, max_depth=11, n_estimators=180;, score=-0.023 total time=   1.2s
[CV 1/5] END gamma=0.4, learning_rate=0.19, max_depth=13, n_estimators=100;, score=-0.017 total time=   0.7s
[CV 2/5] END gamma=0.4, learning_rate=0.19, max_depth=13, n_estimators=100;, score=-0.014 total time=   0.7s
[CV 3/5] END gamma=0.4, learning_rate=0.19, max_depth=13, n_estimators=100;, score=-0.024 total time=   0.8s
[CV 4/5] END gamma=0.4, learning_rate=0.19, max_depth=13, n_estimators=100;, score=-0.019 total time=   0.7s
[CV 5/5] END gamma=

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat...
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None, ...),
             param_grid={'gamma': [0.0, 0.2, 0.4],
                         'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09, 0.11,
                                           0.13, 0.15, 0.17, 0.19],
                         'max_depth': [3, 5, 7, 9, 11, 13, 15],
                         'n_estimators': range(100, 220, 40)},
             scoring='neg_mean_squared_error', verbose=5)

In [12]:
# get best parameter
grid.best_estimator_.get_params()

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': 0.0,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.07,
 'max_bin': 256,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 3,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 180,
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}

In [16]:
#  Training model with hyperparameter tuning
model = XGBRegressor(n_estimators = 180, learning_rate = 0.07, gamma = 0.0, max_depth = 3  )
model.fit(x_train, y_train)
y_pre = model.predict(x_test)
print('Root mean squared' ,(mean_squared_error(y_pre,y_test )**(1/2)))
print('R2-scored ', r2_score(y_pre, y_test))

Root mean squared 0.0840722137586862
R2-scored  0.9955062762475408
